In [1]:
!pip install rasa_nlu==0.14.4
!pip install rasa_core==0.9.6
!pip install sklearn_crfsuite==0.3.6
!pip install pyowm

     |████████████████████████████████| 153kB 6.7MB/s 
     |████████████████████████████████| 5.4MB 41.5MB/s 
     |████████████████████████████████| 655kB 41.3MB/s 
     |████████████████████████████████| 51kB 21.3MB/s 
     |████████████████████████████████| 81kB 26.1MB/s 
     |████████████████████████████████| 12.8MB 24.0MB/s 
     |████████████████████████████████| 829kB 39.9MB/s 
     |████████████████████████████████| 3.1MB 34.9MB/s 
     |████████████████████████████████| 81kB 14.1MB/s 
     |████████████████████████████████| 174kB 42.1MB/s 
     |████████████████████████████████| 61kB 24.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Stored in directory: /root/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
  Stored in directory: /root/.cache/pip/wheels/57/c0/6c/874eb1b6a5ab9ac5434ab9d2700cf78fed3c151ea33cf668af
Successfully built simplejson future Twisted
ERROR: datascience 0.1

     |████████████████████████████████| 133kB 6.3MB/s 
     |████████████████████████████████| 133kB 39.4MB/s 
     |████████████████████████████████| 49.1MB 53.2MB/s 
     |████████████████████████████████| 61kB 22.2MB/s 
     |████████████████████████████████| 71kB 23.9MB/s 
     |████████████████████████████████| 993kB 37.9MB/s 
     |████████████████████████████████| 71kB 25.0MB/s 
     |████████████████████████████████| 307kB 40.3MB/s 
     |████████████████████████████████| 4.9MB 42.0MB/s 
     |████████████████████████████████| 3.1MB 39.6MB/s 
     |████████████████████████████████| 204kB 38.1MB/s 
     |████████████████████████████████| 890kB 33.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/3b/d4/92/3458f0e1a3e1a9c8935d3b2db89a75d74d8d45601fedec752f
  Stored in directory: /root/.cache/pip/wheels/07/8b/41/8ce98f4737a9ff61b1bf2673f2abfe66a6a43ad6e91d2c9736
  Stored in directory: /root/.cache/pip/wheels/76/11/eb/12113becd46c0e8a70b8a63f9405e46a1f61d4aaa8532d676b
  Stored i

In [0]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.actions.action import Action

import pyowm
from datetime import timedelta

In [3]:
!mkdir data
!wget -nc https://raw.githubusercontent.com/ThomasVialars/DL_NLP/master/Task_3/raza_nlu/nlu-config.yml
!wget -nc https://raw.githubusercontent.com/ThomasVialars/DL_NLP/master/Task_3/raza_nlu/domain.yml
!wget -nc https://raw.githubusercontent.com/ThomasVialars/DL_NLP/master/Task_3/raza_nlu/data/stories.md -O data/stories.md
!wget -nc https://raw.githubusercontent.com/ThomasVialars/DL_NLP/master/Task_3/raza_nlu/data/nlu-data.md -O data/nlu-data.md

--2019-06-04 13:50:22--  https://raw.githubusercontent.com/ThomasVialars/DL_NLP/master/Task_3/raza_nlu/nlu-config.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243 [text/plain]
Saving to: ‘nlu-config.yml’

nlu-config.yml      100%[===================>]     243  --.-KB/s    in 0s      

2019-06-04 13:50:22 (36.2 MB/s) - ‘nlu-config.yml’ saved [243/243]

--2019-06-04 13:50:24--  https://raw.githubusercontent.com/ThomasVialars/DL_NLP/master/Task_3/raza_nlu/domain.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353 [text/plain]
Saving

In [0]:
def weather_at(day, carac):  
  owm = pyowm.OWM('81eafa57483f61466c11276914ba8728')
  thf = owm.three_hours_forecast('Novosibirsk, RU')
  weather = thf.get_weather_at(thf.when_starts('date') + timedelta(days = day))  
  
  if (carac == 'weather'):
    return weather.get_detailed_status()
  if (carac == 'temperarure' or carac == 'degrees'):
    return weather.get_temperature(unit='celsius').get('temp')
  if (carac == 'wind'):
    return weather.get_wind()
  if (carac == 'precipitation' or carac == 'rain'):
    return weather.get_rain()

class ActionGetToday(Action):
    def name(self):
      return 'action_today'

    def run(self, dispatcher, tracker, domain):
      t = tracker.get_slot('carac')
      
      if (t == None):
        message = 'Sorry, I can\'t answer'
      else:
        message = weather_at(0, t)

      dispatcher.utter_message(str(message))
      
class ActionGetTomorrow(Action):
    def name(self):
      return 'action_tomorrow'

    def run(self, dispatcher, tracker, domain):
      t = tracker.get_slot('carac')
      
      if (t == None):
        message = 'Sorry, I can\'t answer'
      else:
        message = weather_at(1, t)

      dispatcher.utter_message(str(message))
      
class ActionGetTwo(Action):
    def name(self):
      return 'action_two'

    def run(self, dispatcher, tracker, domain):
      t = tracker.get_slot('carac')
      
      if (t == None):
        message = 'Sorry, I can\'t answer'
      else:
        message = weather_at(2, t)
      
      dispatcher.utter_message(str(message))

class ActionGetThree(Action):
    def name(self):
      return 'action_three'

    def run(self, dispatcher, tracker, domain):
      t = tracker.get_slot('carac')
      
      if (t == None):
        message = 'Sorry, I can\'t answer'
      else:
        message = weather_at(3, t)
      
      dispatcher.utter_message(str(message))
      
class ActionGetFour(Action):
    def name(self):
      return 'action_four'

    def run(self, dispatcher, tracker, domain):
      t = tracker.get_slot('carac')
      
      if (t == None):
        message = 'Sorry, I can\'t answer'
      else:
        message = weather_at(4, t)

      dispatcher.utter_message(str(message))

In [5]:
training_data = load_data('data/nlu-data.md')
trainer = Trainer(config.load('nlu-config.yml'))
interpreter = trainer.train(training_data)
model_directory = trainer.persist('model/nlu', fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


In [6]:
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.8,
                          nlu_threshold=0.6)
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])
training_data = agent.load_data('data/stories.md')
agent.train(training_data, epochs = 100)
agent.persist('model/dialogue')

Using TensorFlow backend.
Processed trackers: 100%|██████████| 258/258 [00:05<00:00, 48.81it/s, # actions=403]
Processed actions: 403it [00:01, 291.10it/s, # examples=403]
Processed trackers: 100%|██████████| 258/258 [00:04<00:00, 53.33it/s, # actions=403]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 5, 14)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6016      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
Total params: 6,280
Trainable params: 6,280
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
403/403 [==============================] - 0s 1ms/step - loss: 1.9873 - acc: 0.4020
Epoch 2/100
403/403 [==============================] - 0s 182us/step - loss: 1.8378 - acc: 0.5434
Epoch 3/100
403/403 [==============================] - 0s 196us/step - loss: 1

In [7]:
interpreter = NaturalLanguageInterpreter.create(model_directory)
agent = Agent.load('model/dialogue', interpreter=interpreter)

/usr/local/lib/python3.6/dist-packages/rasa_nlu/extractors/entity_synonyms.py:85: UserWarning: Failed to load synonyms file from '/content/model/nlu/default/current/entity_synonyms.json'
  "".format(entity_synonyms_file))


In [8]:
responses = agent.handle_message('What is the weather like today?')
for response in responses:
    print(response["text"])

light rain


In [9]:
responses = agent.handle_message('How is the weather tomorrow?')
for response in responses:
    print(response["text"])

broken clouds


In [10]:
responses = agent.handle_message('How much degrees outside in four days?')
for response in responses:
    print(response["text"])

14.35


In [11]:
responses = agent.handle_message('How did Game of Thrones end?')
for response in responses:
    print(response["text"])

Sorry, I didn’t understand that.
